## Creacion de Modelos generales


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Carga de datos
comb_res_1 = pd.read_csv('/content/num_monsters_1_combat_results.csv')
comb_res_2 = pd.read_csv('/content/num_monsters_2_combat_results.csv')
comb_res_3 = pd.read_csv('/content/num_monsters_3_combat_results.csv')
comb_res_1.head()

,Unnamed: 0,pc1_class,pc1_level,pc1_hp_max,pc1_ac,pc1_STR,pc1_DEX,pc1_CON,pc1_INT,pc1_WIS,...,monster7_STR,monster7_DEX,monster7_CON,monster7_INT,monster7_WIS,monster7_CHA,winner,not_conscious_players_ratio,party_hp_ratio,difficulty
0,0,Bard,1,8,10,-1,-1,0,-1,-1,...,0,0,0,0,0,0,monsters,1.0,0.00,9
1,1,Bard,1,7,11,0,0,-1,3,0,...,0,0,0,0,0,0,monsters,1.0,0.00,9
2,2,Bard,1,8,12,3,1,0,-1,2,...,0,0,0,0,0,0,monsters,1.0,0.00,9
3,3,Bard,1,8,14,1,3,0,2,0,...,0,0,0,0,0,0,party,0.0,0.75,2
4,4,Bard,1,9,11,2,0,1,1,2,...,0,0,0,0,0,0,party,0.0,1.00,0


In [6]:
for col in comb_res_1.columns:
  print(col)

Unnamed: 0
pc1_class
pc1_level
pc1_hp_max
pc1_ac
pc1_STR
pc1_DEX
pc1_CON
pc1_INT
pc1_WIS
pc1_CHA
pc2_class
pc2_level
pc2_hp_max
pc2_ac
pc2_STR
pc2_DEX
pc2_CON
pc2_INT
pc2_WIS
pc2_CHA
pc3_class
pc3_level
pc3_hp_max
pc3_ac
pc3_STR
pc3_DEX
pc3_CON
pc3_INT
pc3_WIS
pc3_CHA
pc4_class
pc4_level
pc4_hp_max
pc4_ac
pc4_STR
pc4_DEX
pc4_CON
pc4_INT
pc4_WIS
pc4_CHA
pc5_class
pc5_level
pc5_hp_max
pc5_ac
pc5_STR
pc5_DEX
pc5_CON
pc5_INT
pc5_WIS
pc5_CHA
pc6_class
pc6_level
pc6_hp_max
pc6_ac
pc6_STR
pc6_DEX
pc6_CON
pc6_INT
pc6_WIS
pc6_CHA
pc7_class
pc7_level
pc7_hp_max
pc7_ac
pc7_STR
pc7_DEX
pc7_CON
pc7_INT
pc7_WIS
pc7_CHA
monster1_name
monster1_cr
monster1_hp_max
monster1_ac
monster1_STR
monster1_DEX
monster1_CON
monster1_INT
monster1_WIS
monster1_CHA
monster2_name
monster2_cr
monster2_hp_max
monster2_ac
monster2_STR
monster2_DEX
monster2_CON
monster2_INT
monster2_WIS
monster2_CHA
monster3_name
monster3_cr
monster3_hp_max
monster3_ac
monster3_STR
monster3_DEX
monster3_CON
monster3_INT
monster3_WIS
mons

In [12]:
comb_res_1['difficulty'].unique()

array([9, 2, 0, 3, 4, 5, 1, 7, 6])

Tenemos cargado nuestro dataframe con la informacion de las simulaciones de combate. Entonces, para dejar claro nuestro objetivo es predicir la dificultad de combate de una party con un monstruo.

Las dificultades son:
 - Facil
 - Normal
 - Dificl
 - Mortal

In [15]:
def entrenar_modelo_clasificacion(df, target_col, modelo, test_size=0.2, random_state=42):
    """
    Entrena un modelo de clasificación multiclase con codificación automática de variables categóricas.

    Parámetros:
    - df: DataFrame completo
    - target_col: nombre de la columna objetivo
    - modelo: clasificador (LogisticRegression(), RandomForestClassifier(), etc.)
    - test_size: proporción del dataset para prueba
    - random_state: semilla para reproducibilidad

    Retorna:
    - pipeline entrenado
    - predicciones
    - clasificación y matriz de confusión impresas
    """

    # Eliminar columnas no necesarias
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    # Separar X e y
    X = df.drop(columns=[target_col])
    y = df[target_col].astype(str)

    # Identificar columnas categóricas y numéricas
    categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
    numerical_cols = [col for col in X.columns if X[col].dtype in ["int64", "float64"]]

    # Preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
        ],
        remainder="passthrough"
    )

    # Pipeline
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", modelo)
    ])

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Entrenamiento
    pipeline.fit(X_train, y_train)

    # Predicción
    y_pred = pipeline.predict(X_test)

    # Evaluación
    print("Reporte de Clasificación:")
    print(classification_report(y_test, y_pred))
    print("Matriz de Confusión:")
    print(confusion_matrix(y_test, y_pred))

    return pipeline, y_pred


In [16]:
modelo_lr_1, pred_lr_1 = entrenar_modelo_clasificacion(comb_res_1, "difficulty", LogisticRegression(max_iter=1000,class_weight="balanced",random_state=42))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. 

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.83      0.70      0.76       376
           1       0.40      0.40      0.40       143
           2       0.28      0.56      0.37        32
           3       0.09      0.29      0.14         7
           4       0.33      0.25      0.29         4
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         2
           7       0.50      0.40      0.44         5
           9       1.00      1.00      1.00        19

    accuracy                           0.62       588
   macro avg       0.38      0.40      0.38       588
weighted avg       0.68      0.62      0.64       588

Matriz de Confusión:
[[265  81  16   5   1   0   7   1   0]
 [ 52  57  24   6   0   0   4   0   0]
 [  2   3  18   6   0   0   2   1   0]
 [  0   0   4   2   1   0   0   0   0]
 [  0   0   0   2   1   1   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   1   1 

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.85      0.76      7671
           1       0.84      0.58      0.69     11338
           2       0.52      0.82      0.63      2251
           3       0.67      0.82      0.73       617
           4       0.55      0.79      0.65       131
           5       0.77      0.84      0.80        70
           6       0.76      0.99      0.86       243
           7       0.99      0.99      0.99       214
           9       1.00      1.00      1.00      1867

    accuracy                           0.74     24402
   macro avg       0.75      0.85      0.79     24402
weighted avg       0.77      0.74      0.73     24402

Matriz de Confusión:
[[6520 1086   64    0    0    0    1    0    0]
 [3072 6581 1644    0    0    0   41    0    0]
 [   0  126 1854  242    0    0   29    0    0]
 [   0    0   35  504   75    0    3    0    0]
 [   0    0    0    9  104   18    0    0    0]
 [   0  

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values